In [5]:
import pandas as pd
import json
import os
import re

def preprocess_cosmetic_data():
    """
    올리브영 제품 데이터와 성분 데이터베이스를 결합하여
    머신러닝 학습에 적합한 형태로 전처리하고 CSV 파일로 저장하는 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        coos_df = pd.read_csv('coos_ingredient_database.csv')
        
        json_files = [
            'oliveyoung_로션_raw_limited.json',
            'oliveyoung_미스트_오일_raw_limited.json',
            'oliveyoung_스킨_토너_raw_limited.json',
            'oliveyoung_에센스_세럼_앰플_raw_limited.json',
            'oliveyoung_크림_raw_limited.json'
        ]

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 파일명에서 '카테고리' 정보 추출
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)
        
        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 데이터 전처리 ---
        print("데이터 전처리를 시작합니다...")

        # 브랜드명, 제품명 분리 함수
        def extract_brand_and_clean_name(name):
            brand_match = re.match(r'\[([^\]]+)\]\s*(.*)', name)
            if brand_match:
                parts = brand_match.group(2).split()
                if parts:
                    return parts[0], ' '.join(parts[1:])
                else:
                    return brand_match.group(1), ""
            else:
                parts = name.split()
                if len(parts) > 1:
                    return parts[0], ' '.join(parts[1:])
                else:
                    return name, ""

        oliveyoung_df[['브랜드명', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_brand_and_clean_name(x))
        )

        # 성분 데이터베이스 set 생성 (빠른 탐색용)
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        # 성분 리스트 정제 함수
        def clean_and_normalize_ingredients(ingredient_list):
            cleaned_ingredients = set()
            pattern = re.compile(r'\s*\([^)]*\)|\[[^\]]*\]|\*|\'|"|\s+ppm|\s+ppb|\d+ml\+\d+ml', re.IGNORECASE)
            
            for ing_raw in ingredient_list:
                ing_cleaned = pattern.sub('', ing_raw).strip()
                
                if not ing_cleaned or ing_cleaned.isdigit():
                    continue

                if ing_cleaned in master_ingredients_set:
                    cleaned_ingredients.add(ing_cleaned)
            return list(cleaned_ingredients)

        oliveyoung_df['성분리스트_정리'] = oliveyoung_df['성분리스트'].apply(clean_and_normalize_ingredients)
        print("브랜드명, 제품명, 성분리스트 정제가 완료되었습니다.")

        # --- 3. Feature Matrix 생성 ---
        print("Feature Matrix 생성을 시작합니다...")
        all_unique_ingredients = sorted(list(master_ingredients_set))
        
        feature_df = pd.DataFrame(0, index=oliveyoung_df.index, columns=all_unique_ingredients)

        for index, row in oliveyoung_df.iterrows():
            for ingredient in row['성분리스트_정리']:
                if ingredient in feature_df.columns:
                    feature_df.loc[index, ingredient] = 1
        
        final_df = pd.concat([
            oliveyoung_df[['브랜드명', '제품명', '카테고리']],
            feature_df
        ], axis=1)
        print("Feature Matrix 생성이 완료되었습니다.")

        # --- 4. 결과 저장 ---
        output_filename = 'preprocessed_cosmetics_data.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        print(final_df.head())

    except FileNotFoundError as e:
        print(f"오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"데이터 처리 중 오류가 발생했습니다: {e}")

preprocess_cosmetic_data()

데이터 로딩을 시작합니다...
총 2187개의 제품 데이터를 로드했습니다.
데이터 전처리를 시작합니다...
브랜드명, 제품명, 성분리스트 정제가 완료되었습니다.
Feature Matrix 생성을 시작합니다...
Feature Matrix 생성이 완료되었습니다.

✨ 모든 작업이 완료되었습니다! 결과가 'preprocessed_cosmetics_data.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
     브랜드명                                                제품명 카테고리   γ-터피넨  \
0  아이디얼포맨              [1등올인원] 아이디얼포맨 퍼펙트 올인원 기획세트(+100ml증정)   로션       0   
1  온그리디언츠   [화잘먹로션/속광케어] 온그리디언츠 스킨 베리어 카밍 로션 기획 (220ml+80ml)   로션       0   
2    피지오겔                피지오겔 DMT 페이셜 로션 200ml 기획 (+로션 50ml)   로션       0   
3    에스트라  에스트라 아토베리어365 로션 150ml 기획 (하이드로에센스 25ml+세라-히알 ...   로션       0   
4     닥터지  [리뉴얼/트러블/면도후] 닥터지 레드 블레미쉬 포 맨 진정 올인원 150ml 2종 ...   로션       0   

   (+/-)-테트라하이드로퍼푸릴-(R)-2-[4-(6-클로로퀴노살린-2-일옥시)페닐옥시]프로피오네이트  \
0                                                  0         
1                                                  0         
2                                                  0         
3                                                  0       

In [6]:
import pandas as pd
import json
import os
import re

def preprocess_to_string_efficiently():
    """
    올리브영 제품 데이터와 성분 데이터베이스를 결합하여,
    정제된 성분 리스트를 하나의 문자열로 만들어 CSV로 저장하는 효율적인 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        coos_df = pd.read_csv('coos_ingredient_database.csv')
        
        json_files = [f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')]
        if not json_files:
            print("오류: 'oliveyoung_'으로 시작하는 JSON 파일을 찾을 수 없습니다.")
            return

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)
        
        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 브랜드명 및 제품명 정제 ---
        print("브랜드명과 제품명 정제를 시작합니다...")

        def extract_brand_and_clean_name(name):
            if not isinstance(name, str): return "", ""
            brand_match = re.match(r'\[([^\]]+)\]\s*(.*)', name)
            if brand_match:
                parts = brand_match.group(2).split()
                return (parts[0], ' '.join(parts[1:])) if parts else (brand_match.group(1), "")
            else:
                parts = name.split()
                return (parts[0], ' '.join(parts[1:])) if len(parts) > 1 else (name, "")

        oliveyoung_df[['브랜드명', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_brand_and_clean_name(x))
        )

        # --- 3. 성분 리스트 정제 ---
        print("성분 리스트 정제를 시작합니다...")
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        def clean_and_normalize_ingredients(ingredient_list):
            if not isinstance(ingredient_list, list): return []
            cleaned_ingredients = set()
            for ing_raw in ingredient_list:
                if not isinstance(ing_raw, str): continue
                
                temp_ing = re.sub(r'\[[^\]]+\]|\([^)]+\)|\*|"|\'|\s+ppm|\s+ppb|\d+ml\+\d+ml', '', ing_raw, flags=re.IGNORECASE)
                tokens = temp_ing.split()
                
                for token in tokens:
                    token = token.strip()
                    if token and not token.isdigit() and token in master_ingredients_set:
                        cleaned_ingredients.add(token)
            return sorted(list(cleaned_ingredients)) # 일관된 순서를 위해 정렬

        oliveyoung_df['성분리스트_정리'] = oliveyoung_df['성분리스트'].apply(clean_and_normalize_ingredients)
        print("성분 리스트 정제가 완료되었습니다.")

        # --- 4. 정제된 성분을 하나의 문자열로 합치기 ---
        print("정제된 성분들을 하나의 문자열로 합치는 중입니다...")
        oliveyoung_df['성분_문자열'] = oliveyoung_df['성분리스트_정리'].apply(lambda x: ' '.join(x))

        # --- 5. 최종 데이터프레임 생성 및 저장 ---
        final_df = oliveyoung_df[['브랜드명', '제품명', '카테고리', '성분_문자열']]
        
        output_filename = 'processed_cosmetics_string.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        print(final_df.head())

    except FileNotFoundError as e:
        print(f"\n오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"\n데이터 처리 중 오류가 발생했습니다: {e}")

preprocess_to_string_efficiently()

데이터 로딩을 시작합니다...
총 2287개의 제품 데이터를 로드했습니다.
브랜드명과 제품명 정제를 시작합니다...
성분 리스트 정제를 시작합니다...
성분 리스트 정제가 완료되었습니다.
정제된 성분들을 하나의 문자열로 합치는 중입니다...

✨ 모든 작업이 완료되었습니다! 결과가 'processed_cosmetics_string.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
   브랜드명                                                제품명       카테고리  \
0   웰라쥬  [10월 올영픽/150ml한정기획+마스크] 웰라쥬 리얼 히알루로닉 블루 100 앰플...  에센스_세럼_앰플   
1   메디힐       [단독/1+1] 메디힐 마데카소사이드 흔적 리페어 세럼 40+40ml 더블 기획  에센스_세럼_앰플   
2  브링그린                      브링그린 징크테카 트러블 세럼 (대용량/콜라보/기획)  에센스_세럼_앰플   
3    구달   [리뉴얼] 구달 청귤 비타C 잡티케어 세럼 알파 50ml 리필 기획 (+크림 10ml)  에센스_세럼_앰플   
4  메디큐브  [10월 올영픽/1등 미백앰플] 메디큐브 PDRN 핑크 앰플 30ml 리필기획 (본...  에센스_세럼_앰플   

                                              성분_문자열  
0  갈랑가갈랑갈추출물 글라이신 글라이코실트레할로오스 글리세레스-26 글리세린 글리세릴아...  
1  광곽향오일 글리세린 글리세릴폴리메타크릴레이트 나이아신아마이드 다마스크장미꽃수 다이페...  
2  4-터피네올 글라이코리피드 나이아신아마이드 다이소듐이디티에이 메틸글루세스-20 메틸...  
3  3-O-에틸아스코빅애씨드 감초뿌리추출물 글루타티온 글리세레스-26 글리세린 나이아신...  
4  글리세레스-26 글리세린 글리세릴아크릴레이트/아크릴릭애씨드코폴리머 나이아신아마이드 ...  


In [7]:
import pandas as pd
import json
import os
import re

def final_advanced_preprocessing_with_custom_keywords():
    """
    사용자가 제공한 샘플 데이터를 기반으로 추출한 키워드를 사용하여
    제품명 정제 로직을 고도화하고, 최종 CSV로 저장하는 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        coos_df = pd.read_csv('coos_ingredient_database.csv')

        json_files = [f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')]
        if not json_files:
            print("오류: 'oliveyoung_'으로 시작하는 JSON 파일을 찾을 수 없습니다.")
            return

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)

        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 제품명 및 브랜드명 정제 (사용자 데이터 기반 고도화) ---
        print("사용자 샘플 기반의 고도화된 로직으로 제품명과 브랜드명 정제를 시작합니다...")

        def extract_and_clean_product_info(original_name):
            if not isinstance(original_name, str):
                return "", "", ""

            # 브랜드명 후보 추출
            temp_name = original_name
            brand_match = re.match(r'\[[^\]]+\]\s*([^\s]+)', temp_name)
            if brand_match:
                brand_cleaned = brand_match.group(1)
            else:
                brand_cleaned = temp_name.split()[0] if temp_name.split() else ""

            # 제품명 정제 시작
            cleaned_name = re.sub(r'\[[^\]]+\]', '', temp_name)
            cleaned_name = re.sub(r'\([^)]+\)', '', cleaned_name)
            cleaned_name = re.sub(r'\s*\+.*', '', cleaned_name)

            cleaned_name = cleaned_name.replace(brand_cleaned, '', 1).strip()

            cleaned_name = re.sub(r'\b\d+(\.\d+)?\s*(ml|g|L|p|P)\b', '', cleaned_name, flags=re.I)
            cleaned_name = re.sub(r'\b\d+\s*[입매개eaEA]+\b', '', cleaned_name, flags=re.I)
            cleaned_name = re.sub(r'\b(\d+\+\d+|\d\+\d)\b', '', cleaned_name)

            promo_keywords = [
                '기획', '한정', '증정', '더블', '대용량', '리필', '올영픽', '단독', '콜라보', '세트', '리뉴얼',
                '알파', 'PICK', 'UP', '특가', '에디션', '아따맘마', '춘구마', '가나디', '포켓몬', '1등',
                '화잘먹', '흔적', '진정', '모공', '탄력', '수분', '미백', '잡티', '광채'
            ]
            for keyword in promo_keywords:
                cleaned_name = re.sub(r'\b' + re.escape(keyword) + r'\b', '', cleaned_name)

            cleaned_name = re.sub(r'\b\d+\b', '', cleaned_name)
            cleaned_name = ' '.join(cleaned_name.split()).strip()

            if not cleaned_name:
                cleaned_name = ' '.join(original_name.replace(brand_cleaned, '').split()).strip()

            return original_name, brand_cleaned, cleaned_name

        oliveyoung_df[['제품명_원본', '브랜드명_정리', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_and_clean_product_info(x))
        )
        print("제품명 정제가 완료되었습니다.")

        # --- 3. 성분 리스트 정제 및 문자열 변환 ---
        print("성분 리스트 정제 및 문자열 변환을 시작합니다...")
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        def clean_and_stringify_ingredients(ingredient_list):
            if not isinstance(ingredient_list, list): return ""
            cleaned_ingredients = set()
            for ing_raw in ingredient_list:
                if not isinstance(ing_raw, str): continue
                temp_ing = re.sub(r'\[[^\]]+\]|\([^)]+\)|\*|"|\'|\s+ppm|\s+ppb|\d+ml\+\d+ml', '', ing_raw, flags=re.I)
                tokens = temp_ing.split()
                for token in tokens:
                    token = token.strip()
                    if token and not token.isdigit() and token in master_ingredients_set:
                        cleaned_ingredients.add(token)
            return ' '.join(sorted(list(cleaned_ingredients)))

        oliveyoung_df['성분_문자열'] = oliveyoung_df['성분리스트'].apply(clean_and_stringify_ingredients)
        print("성분 데이터 처리가 완료되었습니다.")

        # --- 4. 최종 데이터프레임 생성 및 저장 ---
        final_df = oliveyoung_df[[
            '브랜드명_정리',
            '제품명_정리',
            '카테고리',
            '성분_문자열',
            '제품명_원본'
        ]]

        output_filename = 'processed_cosmetics_final.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        print(final_df.head())

    except FileNotFoundError as e:
        print(f"\n오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"\n데이터 처리 중 오류가 발생했습니다: {e}")

final_advanced_preprocessing_with_custom_keywords()

데이터 로딩을 시작합니다...
총 2287개의 제품 데이터를 로드했습니다.
사용자 샘플 기반의 고도화된 로직으로 제품명과 브랜드명 정제를 시작합니다...
제품명 정제가 완료되었습니다.
성분 리스트 정제 및 문자열 변환을 시작합니다...
성분 데이터 처리가 완료되었습니다.

✨ 모든 작업이 완료되었습니다! 결과가 'processed_cosmetics_final.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
  브랜드명_정리           제품명_정리       카테고리  \
0     웰라쥬   리얼 히알루로닉 블루 앰플  에센스_세럼_앰플   
1     메디힐   마데카소사이드 리페어 세럼  에센스_세럼_앰플   
2    브링그린      징크테카 트러블 세럼  에센스_세럼_앰플   
3      구달   청귤 비타C 잡티케어 세럼  에센스_세럼_앰플   
4    메디큐브  PDRN 핑크 앰플 리필기획  에센스_세럼_앰플   

                                              성분_문자열  \
0  갈랑가갈랑갈추출물 글라이신 글라이코실트레할로오스 글리세레스-26 글리세린 글리세릴아...   
1  광곽향오일 글리세린 글리세릴폴리메타크릴레이트 나이아신아마이드 다마스크장미꽃수 다이페...   
2  4-터피네올 글라이코리피드 나이아신아마이드 다이소듐이디티에이 메틸글루세스-20 메틸...   
3  3-O-에틸아스코빅애씨드 감초뿌리추출물 글루타티온 글리세레스-26 글리세린 나이아신...   
4  글리세레스-26 글리세린 글리세릴아크릴레이트/아크릴릭애씨드코폴리머 나이아신아마이드 ...   

                                              제품명_원본  
0  [10월 올영픽/150ml한정기획+마스크] 웰라쥬 리얼 히알루로닉 블루 100 앰플...  
1       [단독/1+1] 메디힐 마데카소사이드 흔적 리페어 세럼 40+40ml 더블 기획  
2                

In [8]:
import pandas as pd
import json
import os
import re

def final_advanced_preprocessing_with_custom_keywords():
    """
    제공된 'coos_ingredient_database.csv' 및 'oliveyoung_*.json' 파일들을 기반으로
    제품명 정제 로직을 고도화하고, 성분 리스트를 정제하여 최종 CSV로 저장하는 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        
        # 성분 데이터베이스 로드
        coos_df = pd.read_csv('coos_ingredient_database.csv')

        # 'oliveyoung_'으로 시작하는 모든 JSON 파일 찾기
        json_files = [f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')]
        if not json_files:
            print("오류: 'oliveyoung_'으로 시작하는 JSON 파일을 찾을 수 없습니다. (예: oliveyoung_크림_raw_limited.json)")
            return

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 파일명에서 카테고리 추출 (예: oliveyoung_로션_raw_limited.json -> 로션)
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)

        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 제품명 및 브랜드명 정제 (고도화된 로직 적용) ---
        print("고도화된 로직으로 제품명과 브랜드명 정제를 시작합니다...")

        def extract_and_clean_product_info(original_name):
            if not isinstance(original_name, str):
                return "", "", ""

            # 1. 브랜드명 후보 추출
            temp_name = original_name
            # [특수문자]로 시작하는 경우 [특수문자] 다음에 오는 첫 번째 단어를 브랜드로 간주
            brand_match = re.match(r'\[[^\]]+\]\s*([^\s]+)', temp_name)
            if brand_match:
                brand_cleaned = brand_match.group(1)
            else:
                # 일반적인 경우 첫 단어를 브랜드로 간주
                brand_cleaned = temp_name.split()[0] if temp_name.split() else ""

            # 2. 제품명 정제 시작 (필요 없는 정보 제거)
            cleaned_name = temp_name
            
            # 괄호와 대괄호 안의 내용 제거
            cleaned_name = re.sub(r'\[[^\]]+\]', '', cleaned_name)
            cleaned_name = re.sub(r'\([^)]+\)', '', cleaned_name)
            
            # '+' 기호 뒤의 내용 제거 (대부분 추가 증정/기획 관련 내용)
            cleaned_name = re.sub(r'\s*\+.*', '', cleaned_name)

            # 브랜드명 제거
            cleaned_name = cleaned_name.replace(brand_cleaned, '', 1).strip()
            
            # 용량 정보 제거 (ml, g, L, p, P, 입, 매, 개, ea, EA)
            cleaned_name = re.sub(r'\b\d+(\.\d+)?\s*(ml|g|L|p|P)\b', '', cleaned_name, flags=re.I)
            cleaned_name = re.sub(r'\b\d+\s*[입매개eaEA]+\b', '', cleaned_name, flags=re.I)
            
            # 용량 합산 표기 제거 (예: 100+100, 1+1)
            cleaned_name = re.sub(r'\b(\d+\+\d+|\d\+\d)\b', '', cleaned_name)

            # 고도화된 프로모션/키워드 제거 리스트
            promo_keywords = [
                '기획', '한정', '증정', '더블', '대용량', '리필', '올영픽', '단독', '콜라보', '세트', '리뉴얼',
                '알파', 'PICK', 'UP', '특가', '에디션', '아따맘마', '춘구마', '가나디', '포켓몬', '1등',
                '화잘먹', '흔적', '진정', '모공', '탄력', '수분', '미백', '잡티', '광채', 'SPF\s*\d+', 'PA\s*\+\s*\+'
            ]
            for keyword in promo_keywords:
                # 단어 경계를 사용하여 정확히 일치하는 단어만 제거
                cleaned_name = re.sub(r'\b' + re.escape(keyword) + r'\b', '', cleaned_name, flags=re.I)

            # 단일 숫자 제거 (위에서 처리되지 않은 경우)
            cleaned_name = re.sub(r'\b\d+\b', '', cleaned_name)
            
            # 과도한 공백 제거 및 최종 정리
            cleaned_name = ' '.join(cleaned_name.split()).strip()

            # 최종 제품명이 비어있을 경우, 브랜드명 제거 후 원본에서 최대한 복구
            if not cleaned_name:
                cleaned_name = ' '.join(original_name.replace(brand_cleaned, '').split()).strip()

            return original_name, brand_cleaned, cleaned_name

        # 정제 함수를 DataFrame에 적용
        oliveyoung_df[['제품명_원본', '브랜드명_정리', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_and_clean_product_info(x))
        )
        print("제품명 정제가 완료되었습니다.")

        # --- 3. 성분 리스트 정제 및 문자열 변환 ---
        print("성분 리스트 정제 및 문자열 변환을 시작합니다...")
        
        # 성분 데이터베이스에서 유효한 성분명 세트 생성
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        def clean_and_stringify_ingredients(ingredient_list):
            if not isinstance(ingredient_list, list): return ""
            cleaned_ingredients = set()
            
            for ing_raw in ingredient_list:
                if not isinstance(ing_raw, str): continue
                
                # 괄호, 특수문자, 용량 정보, 따옴표 등을 제거
                temp_ing = re.sub(r'\[[^\]]+\]|\([^)]+\)|\*|"|\'|\s+ppm|\s+ppb|\d+ml\+\d+ml', '', ing_raw, flags=re.I)
                
                # 공백 기준으로 토큰 분리 (성분 데이터에 따라 여러 성분이 한 문자열에 있을 수 있음)
                tokens = temp_ing.split()
                
                for token in tokens:
                    token = token.strip()
                    # 토큰이 비어있지 않고, 숫자가 아니며, 마스터 성분 목록에 있을 경우에만 추가
                    if token and not token.isdigit() and token in master_ingredients_set:
                        cleaned_ingredients.add(token)
                        
            # 성분명을 알파벳 순서로 정렬하여 하나의 공백으로 구분된 문자열로 반환
            return ' '.join(sorted(list(cleaned_ingredients)))

        oliveyoung_df['성분_문자열'] = oliveyoung_df['성분리스트'].apply(clean_and_stringify_ingredients)
        print("성분 데이터 처리가 완료되었습니다.")

        # --- 4. 최종 데이터프레임 생성 및 저장 ---
        final_df = oliveyoung_df[[
            '브랜드명_정리',
            '제품명_정리',
            '카테고리',
            '성분_문자열',
            '제품명_원본'
        ]].copy() # 불필요한 SettingWithCopyWarning 방지

        output_filename = 'processed_cosmetics_final_2.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        # 인덱스 없이 깔끔하게 출력
        print(final_df.head().to_string(index=False))

    except FileNotFoundError as e:
        print(f"\n오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"\n데이터 처리 중 오류가 발생했습니다: {e}")

# 함수 실행
final_advanced_preprocessing_with_custom_keywords()


데이터 로딩을 시작합니다...
총 2187개의 제품 데이터를 로드했습니다.
고도화된 로직으로 제품명과 브랜드명 정제를 시작합니다...
제품명 정제가 완료되었습니다.
성분 리스트 정제 및 문자열 변환을 시작합니다...
성분 데이터 처리가 완료되었습니다.

✨ 모든 작업이 완료되었습니다! 결과가 'processed_cosmetics_final_2.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
브랜드명_정리          제품명_정리      카테고리                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    성분

In [9]:
import os
import json
import re

def check_json_file_counts():
    """
    현재 디렉토리에서 'oliveyoung_'으로 시작하는 JSON 파일을 찾아
    각 파일에 포함된 제품(아이템)의 개수를 출력합니다.
    """
    print("--- JSON 파일별 제품 개수 분석을 시작합니다 ---")
    
    # 1. 파일 찾기: 'oliveyoung_'으로 시작하고 '.json'으로 끝나는 파일 목록 스캔
    json_files = [f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')]
    
    if not json_files:
        print("경고: 'oliveyoung_'으로 시작하는 JSON 파일을 현재 디렉토리에서 찾을 수 없습니다.")
        return

    total_count = 0
    
    print("\n[카테고리별 제품 개수]")
    
    for file in json_files:
        try:
            # 2. 파일 로드 및 카테고리 추출
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # 파일명에서 카테고리 추출 (예: oliveyoung_크림_raw_limited.json -> 크림)
                category_raw = os.path.basename(file)
                category = category_raw.replace('oliveyoung_', '').replace('_raw_limited.json', '')
                
                # 3. 제품 개수 확인 및 출력
                count = len(data)
                print(f"  - {category:<10} ({category_raw}): {count:5,}개") 
                total_count += count
                
        except json.JSONDecodeError:
            print(f"오류: {file} 파일이 유효한 JSON 형식이 아닙니다. 건너뜁니다.")
        except Exception as e:
            print(f"오류: {file} 처리 중 예상치 못한 에러 발생 - {e}")

    print(f"\n=====================================")
    print(f"✅ 총 로드된 제품 데이터 개수: {total_count:,}개")
    print(f"=====================================")

# 함수 실행
check_json_file_counts()


--- JSON 파일별 제품 개수 분석을 시작합니다 ---

[카테고리별 제품 개수]
  - 에센스_세럼_앰플  (oliveyoung_에센스_세럼_앰플_raw_limited.json):   720개
  - 스킨_토너      (oliveyoung_스킨_토너_raw_limited.json):   386개
  - 로션         (oliveyoung_로션_raw_limited.json):   252개
  - 미스트_오일     (oliveyoung_미스트_오일_raw_limited.json):   109개
  - 크림         (oliveyoung_크림_raw_limited.json):   720개

✅ 총 로드된 제품 데이터 개수: 2,187개


In [10]:
import os
import json
import re

def check_json_file_counts():
    """
    현재 디렉토리에서 'oliveyoung_'으로 시작하는 JSON 파일을 찾아
    각 파일에 포함된 제품(아이템)의 개수를 출력하고, 오류 발생 파일을 카운트합니다.
    오류 발생 시 해당 파일의 순서(인덱스)도 함께 출력합니다.
    """
    print("--- JSON 파일별 제품 개수 분석을 시작합니다 ---")
    
    # 1. 파일 찾기: 'oliveyoung_'으로 시작하고 '.json'으로 끝나는 파일 목록 스캔
    json_files = sorted([f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')])
    
    if not json_files:
        print("경고: 'oliveyoung_'으로 시작하는 JSON 파일을 현재 디렉토리에서 찾을 수 없습니다.")
        return

    total_count = 0
    error_count = 0
    error_details = [] # 오류 파일 이름과 순서를 저장할 리스트

    print(f"총 {len(json_files)}개의 파일을 스캔합니다.")
    print("\n[카테고리별 제품 개수]")
    
    # 파일 순서를 확인하기 위해 enumerate 사용
    for index, file in enumerate(json_files):
        file_number = index + 1 # 1부터 시작하는 순서
        
        try:
            # 2. 파일 로드 및 카테고리 추출
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # 파일명에서 카테고리 추출
                category_raw = os.path.basename(file)
                category = category_raw.replace('oliveyoung_', '').replace('_raw_limited.json', '')
                
                # 3. 제품 개수 확인 및 출력
                count = len(data)
                print(f"  [{file_number:02d}] {category:<10} ({category_raw}): {count:5,}개") 
                total_count += count
                
        except json.JSONDecodeError:
            error_count += 1
            error_details.append((file_number, file, "유효한 JSON 형식이 아님"))
            print(f"  [{file_number:02d}] 오류 발생: {file} - 유효한 JSON 형식이 아닙니다.")
        except Exception as e:
            error_count += 1
            error_details.append((file_number, file, str(e)))
            print(f"  [{file_number:02d}] 오류 발생: {file} 처리 중 예상치 못한 에러 발생 ({e})")

    print(f"\n=====================================")
    print(f"✅ 총 로드된 제품 데이터 개수: {total_count:,}개")
    
    # 오류 개수 및 상세 정보 출력
    if error_count > 0:
        print(f"🚨 로드 실패 파일 개수: {error_count}개")
        print("\n[오류 발생 파일 상세 정보 (순서, 파일명, 원인)]")
        for num, name, reason in error_details:
            # 오류 파일 순서(num)를 포함하여 출력
            print(f"  - 순서 {num:02d}: {name} (원인: {reason})")
    else:
        print("✨ 모든 파일이 오류 없이 성공적으로 로드되었습니다.")
        
    print(f"=====================================")

# 함수 실행
check_json_file_counts()


--- JSON 파일별 제품 개수 분석을 시작합니다 ---
총 5개의 파일을 스캔합니다.

[카테고리별 제품 개수]
  [01] 로션         (oliveyoung_로션_raw_limited.json):   252개
  [02] 미스트_오일     (oliveyoung_미스트_오일_raw_limited.json):   109개
  [03] 스킨_토너      (oliveyoung_스킨_토너_raw_limited.json):   386개
  [04] 에센스_세럼_앰플  (oliveyoung_에센스_세럼_앰플_raw_limited.json):   720개
  [05] 크림         (oliveyoung_크림_raw_limited.json):   720개

✅ 총 로드된 제품 데이터 개수: 2,187개
✨ 모든 파일이 오류 없이 성공적으로 로드되었습니다.


In [13]:
import pandas as pd
import re
from typing import Set

# 파일로 저장할 때 사용할 파일명
OUTPUT_FILENAME = 'coos_master_ingredients_cleaned.csv'

def clean_coos_ingredient(ing_raw: str) -> str | None:
    """
    COOS 데이터베이스의 원료명에서 불필요한 마킹(괄호 내용, 특수 문자, 용량 등)을 제거하여
    깨끗한 성분명을 추출합니다.
    """
    if not isinstance(ing_raw, str):
        return None
        
    # 괄호 안의 내용 (*, ppm 등) 및 특수 문자, 용량 관련 패턴 제거
    temp_ing = re.sub(r'\[[^\]]+\]|\([^)]+\)|\*|"|\'|\s+ppm|\s+ppb|\d+ml\+\d+ml|\d+\s*ea', '', ing_raw, flags=re.I)
    temp_ing = temp_ing.strip()
    
    # 빈 문자열이거나 숫자만 남은 경우 제외
    return temp_ing if temp_ing and not temp_ing.isdigit() else None

def get_master_ingredients_set(filepath: str = 'coos_ingredient_database.csv') -> Set[str]:
    """
    COOS 성분 데이터베이스 CSV 파일을 로드하고, 원료명을 정제하여 
    마스터 성분 세트(set)를 반환합니다.
    
    주의: 이 함수는 마스터 세트를 반환하며, 파일을 저장하는 기능은 
          __main__ 블록에서 처리합니다.
    """
    print(f"[COOS Processor] '{filepath}' 파일을 로드하여 정제 작업을 시작합니다...")
    try:
        # COOS 데이터 로드
        coos_df = pd.read_csv(filepath)
        
        # '원료명' 컬럼에 정제 함수를 적용하고, 고유한 값만 추출하여 마스터 세트 생성
        coos_cleaned_ingredients = coos_df['원료명'].apply(clean_coos_ingredient).dropna().unique()
        master_ingredients_set = set(coos_cleaned_ingredients)
        
        print(f"✅ [COOS Processor] '{filepath}'에서 총 {len(master_ingredients_set):,}개의 유효 성분명을 로드 및 정제했습니다.")
        return master_ingredients_set
    
    except FileNotFoundError:
        print(f"❌ [COOS Processor] 오류: 파일을 찾을 수 없습니다. '{filepath}' 경로를 확인해주세요.")
        return set()
    except Exception as e:
        print(f"❌ [COOS Processor] 데이터 로딩 또는 정제 중 오류 발생: {e}")
        return set()

# --- 실행만 해도 되도록 추가된 부분 ---
if __name__ == "__main__":
    # 1. 마스터 성분 세트 로드 및 정제
    master_set = get_master_ingredients_set()
    
    if master_set:
        # 2. 정제된 세트를 DataFrame으로 변환
        master_df = pd.DataFrame(list(master_set), columns=['원료명_정제됨'])
        
        # 3. CSV 파일로 저장
        try:
            master_df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
            print(f"\n🎉 [독립 실행 결과] 정제된 마스터 성분 목록을 '{OUTPUT_FILENAME}' 파일로 저장했습니다.")
            print(f"   (총 {len(master_set):,}개 성분)")
        except Exception as e:
            print(f"\n❌ [저장 오류] 정제된 데이터를 CSV 파일로 저장하는 데 실패했습니다: {e}")


[COOS Processor] 'coos_ingredient_database.csv' 파일을 로드하여 정제 작업을 시작합니다...
✅ [COOS Processor] 'coos_ingredient_database.csv'에서 총 25,394개의 유효 성분명을 로드 및 정제했습니다.

🎉 [독립 실행 결과] 정제된 마스터 성분 목록을 'coos_master_ingredients_cleaned.csv' 파일로 저장했습니다.
   (총 25,394개 성분)
